# LangChain `@tool` 데코레이터

In [1]:
from datetime import datetime
from dotenv import load_dotenv
import pytz
from langchain_core.tools import tool  # decorator
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [2]:
# .env 파일에 저장된 api_key를 OS 환경 변수로 로딩
load_dotenv()

True

In [3]:
# OpenAI 클라이언트를 생성
model = ChatOpenAI(model='gpt-4o-mini')

In [20]:
# AI에게 메시지 전달하고 실행(invoke).
messages = [
    SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.'),
    HumanMessage(content='지금 현재 서울 시간?')
]

In [ ]:
ai_message = model.invoke(input=messages)

In [5]:
type(ai_message)

langchain_core.messages.ai.AIMessage

In [6]:
print(ai_message)  # print(ai_message.content)

content='현재 시간 정보를 실시간으로 제공할 수는 없지만, 서울은 한국 표준시(KST, UTC+9)에 위치하고 있습니다. 따라서 현재 시간을 확인하려면 UTC 시간을 기준으로 9시간을 더하면 됩니다. 스마트폰이나 컴퓨터에서 시간을 확인할 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 30, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CGE5XmJfzOTuE4VJAtOuESJ1GoTIz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3470571b-7cd1-4e3b-a6b4-63d7a3340b32-0' usage_metadata={'input_tokens': 30, 'output_tokens': 60, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [7]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 시간 정보를 실시간으로 제공할 수는 없지만, 서울은 한국 표준시(KST, UTC+9)에 위치하고 있습니다. 따라서 현재 시간을 확인하려면 UTC 시간을 기준으로 9시간을 더하면 됩니다. 스마트폰이나 컴퓨터에서 시간을 확인할 수 있습니다.


도구(tool)을 AI에게 제공하고, AI는 tool 목록에 있는 함수 호출(function calling)을 요청해서 에이전트가 함수 호출 결과를 다시 AI에게 전송하면 AI는 함수 호출 결과를 바탕으로 답변을 생성할 수 있음.

사용자 질문(도구 목록 제공) -> AI 도구 호출 요청 -> 사용자 함수 호출 결과 -> AI 답변 생성.

In [21]:
@tool
def get_current_time(timezone: str, location: str) -> str:
    """해당 timezone의 현재 날짜와 시간을 문자열로 리턴.

    Args:
        timezone (str): 타임존. (예: Asia/Seoul)
        location (str): 지역명. 타임존은 모든 도시 이름에 대응되지 않기 때문에 LLM이 답변을 생성할 때 이용하도록 제공.
    Returns: '날짜 시간 타임존(지역명)' 형식의 문자열을 반환. (예: 2025-09-15 15:30:55 Asia/Seoul(부산))
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
    result = f'{now} {timezone}({location})'

    return result

In [22]:
# 도구 목록: @tool 데코레이터가 사용된 함수들의 리스트.
tools = [get_current_time,]

In [23]:
# AI에서 도구 목록의 함수 호출을 요청했을 때 함수 객체를 쉽게 찾기 위해서 dict 선언.
# 함수 이름을 key로, 함수 객체를 value로 갖는 딕셔너리.
tool_dict = {
    'get_current_time': get_current_time,
}

In [24]:
# AI 모델과 도구 목록을 binding(묶어줌).
model_with_tools = model.bind_tools(tools=tools)

In [25]:
messages = [
    SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.'),
    HumanMessage(content='지금 현재 서울 시간?')
]

In [26]:
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='지금 현재 서울 시간?', additional_kwargs={}, response_metadata={})]


In [27]:
# 도구 목록을 가지고 있는 AI 모델을 호출
response = model_with_tools.invoke(input=messages)

In [28]:
type(response)

langchain_core.messages.ai.AIMessage

In [29]:
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_UcZri34ZEEiq55ACEUTGxXNb', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"서울"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 168, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CGEb0rFkJjsL785mz8R6QkWMbvMgP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--d6108be5-239b-40c3-b4c0-4fcfaabd4b0a-0' tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '서울'}, 'id': 'call_UcZri34ZEEiq55ACEUTGxXNb', 'type': 'tool_call'}] usage_metadata={'input_tokens': 168, 'output_tokens': 22, '

In [30]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_UcZri34ZEEiq55ACEUTGxXNb)
 Call ID: call_UcZri34ZEEiq55ACEUTGxXNb
  Args:
    timezone: Asia/Seoul
    location: 서울


In [31]:
messages.append(response)  # 대화 이력 저장: AIMessage 객체를 리스트에 추가!

In [32]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답하는 AI 비서야.
================================ Human Message =================================

지금 현재 서울 시간?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_UcZri34ZEEiq55ACEUTGxXNb)
 Call ID: call_UcZri34ZEEiq55ACEUTGxXNb
  Args:
    timezone: Asia/Seoul
    location: 서울


In [33]:
for tool_call in response.tool_calls:
    fn = tool_dict[tool_call['name']]  # AIMessage의 포함된 함수 이름으로 함수 객체를 찾음.
    tool_msg = fn.invoke(tool_call)  # @tool 데코레이터로 포장 -> invoke가 가능.
    # invoke: tool_call에서 args를 찾고 fn를 호출할 때 아규먼트를 전달. 리턴값을 이용해서 ToolMessage 객체 반환.
    messages.append(tool_msg)

In [34]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답하는 AI 비서야.
================================ Human Message =================================

지금 현재 서울 시간?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_UcZri34ZEEiq55ACEUTGxXNb)
 Call ID: call_UcZri34ZEEiq55ACEUTGxXNb
  Args:
    timezone: Asia/Seoul
    location: 서울
================================= Tool Message =================================
Name: get_current_time

2025-09-16 10:17:05 Asia/Seoul(서울)


In [35]:
ai_message = model_with_tools.invoke(input=messages)

In [36]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 서울 시간은 2025년 9월 16일 10:17:05입니다.


## 2개 이상의 도구 호출을 포함하는 tool_calls 리스트

In [55]:
messages = [
    SystemMessage('사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.'),
    SystemMessage('도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.'),
    HumanMessage('서울, 부산, 런던의 현재 시간은?'),
]

In [56]:
ai_message = model_with_tools.invoke(input=messages)

In [57]:
messages.append(ai_message)

In [58]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.
================================ Human Message =================================

서울, 부산, 런던의 현재 시간은?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_SjfNTNovaKMmeOHql5P8tMXD)
 Call ID: call_SjfNTNovaKMmeOHql5P8tMXD
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_skCI7t0KG2qXhI9m6AfZlV7J)
 Call ID: call_skCI7t0KG2qXhI9m6AfZlV7J
  Args:
    timezone: Asia/Seoul
    location: 부산
  get_current_time (call_2WHWIaenf4y9MQyKItqUPaYX)
 Call ID: call_2WHWIaenf4y9MQyKItqUPaYX
  Args:
    timezone: Europe/London
    location: 런던


In [59]:
for tool_call in ai_message.tool_calls:
    fn = tool_dict[tool_call['name']]
    tool_msg = fn.invoke(tool_call)
    messages.append(tool_msg)

In [60]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.
================================ Human Message =================================

서울, 부산, 런던의 현재 시간은?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_SjfNTNovaKMmeOHql5P8tMXD)
 Call ID: call_SjfNTNovaKMmeOHql5P8tMXD
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_skCI7t0KG2qXhI9m6AfZlV7J)
 Call ID: call_skCI7t0KG2qXhI9m6AfZlV7J
  Args:
    timezone: Asia/Seoul
    location: 부산
  get_current_time (call_2WHWIaenf4y9MQyKItqUPaYX)
 Call ID: call_2WHWIaenf4y9MQyKItqUPaYX
  Args:
    timezone: Europe/London
    location: 런던
================================= Tool Message =================================
Name: get_current_time

2025-09-16 10:52:13 Asia/Seoul(서울)
=

In [61]:
ai_message = model_with_tools.invoke(input=messages)

In [62]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 시간은 다음과 같습니다:

- 서울: 2025-09-16 10:52:13 
- 부산: 2025-09-16 10:52:13 
- 런던: 2025-09-16 02:52:13


# Pydantic

입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확히 표현

In [63]:
def subtract(x: int, y: int) -> int:
    return x - y

In [64]:
subtract(1, 2)

-1

In [65]:
subtract(1.1, 2.5)

-1.4

Python은 파라미터의 타입 검사를 하지 않고, 리턴 타입도 힌트대로 반환하는 것은 아님.

In [66]:
subtract('a', 'b')

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [67]:
class Person:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name

In [68]:
person = Person(id=1, name='오쌤')

In [69]:
print(person.id)

1


In [70]:
person2 = Person(id='abc123', name=123)

In [71]:
print(person2.id)

abc123


Python은 객체를 생성할 때 속성(property)들의 타입 검사를 수행하지 않음.

In [72]:
from pydantic import BaseModel, ValidationError, PositiveInt

In [73]:
class Person(BaseModel):  # Person 클래스는 BaseModel 클래스를 상속(확장).
    id: PositiveInt
    name: str

In [74]:
person3 = Person(id=12345, name='홍길동')

In [75]:
print(person3)

id=12345 name='홍길동'


In [76]:
person3.model_dump()  # 인스턴스의 속성(property) 이름과 값으로 dict를 생성해서 리턴.

{'id': 12345, 'name': '홍길동'}

In [78]:
try:
    person4 = Person(id='abc123', name=1_000)
    print(person4)
except ValidationError as e:
    print(e)

2 validation errors for Person
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
name
  Input should be a valid string [type=string_type, input_value=1000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


In [81]:
person5 = Person(**{'id': 123456, 'name': '길동'})  # **: unpacking 연산자
print(person5)

id=123456 name='길동'
